## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')


### Вывод

#### children
Странные значения -1 и 20:
вероятно -1 подаются в случае если значение было неизвестно, 20 - не совсем ясно. Так или иначе от таких значений желательно избавиться для финальных вопросов.  
#### days_employed: 
по всей видимости при добавлении данных в колонку использовалась разница между текущей временем и датой трудоустройства. В таком случае значение после точки можно отбросить как незначительное, а основную часть взять для удобства по модулю. 
Также в этой колонке, в случае если род деятельности отмечен как пенсионер значение days_employed сильно отличается, возможно был произведен неправильный забор данных.  
Также часть данных в этой колонке пропущено, необходимо обдумать что будет разумнее - дропнуть эти строки или заполнить отсутствующие значения.  
Если в строке пустые значения в days_employed, то они будут пустыми и в total income  
Т.к. ни в запросе банка, ни для ответа на финальные вопросы нет необходимости обрабатывать трудовой стаж, можем опустить эти несостыковки.  
#### dob_years: 
0 в некоторых строках. Колонка не участвует в финальных вопросах, поэтому нулевыми значениями можно принебречь
#### education, education_id: 
в колонке education данные представлены в разных регистрах, однако одинаковые значения в разных регистрах имеют единый education_id, а значит никакой путаницы в данных не предвидется. Можно было бы привести к единому регистру для удобства, но колонка не участвует в финальных подсчетах
#### gender: 
неинтерпритируемое значение - XNA. Колонка не участвует в финальных вопросах, поэтому неинтерпритируемое значение можно принебречь
#### total_income: 
В колонке можно для удобства отбросить дробную часть ( возможно дробная часть образовалось от подсчета дохода после налогов). Также некоторые значения отсутствуют (около 10%), необходимо решить что делать с пропусками.  
#### purpose: 
схожие назначение, имеющие разный тип записи - например:  
'ремонт жилью', 'операции с жильем', 'жилье'   
Категоризация данных по основным направлениям - свадьба, жилье, автомобиль, образование (см. шаг с лемматизацией)

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
df["total_income"] = df["total_income"].fillna(0)
# выделим отдельную колонку без незаполненных данных total_income, для обработки вопросов связанных с доходами
# полученный результат скопируем для избежания дальнейших предупреждений
income_without_null = df[df["total_income"] != 0].copy();
df["days_employed"] = df["days_employed"].fillna(0)
# избавимся от неоднозначных значениях в колонке children, 
# полученный результат скопируем во избежание предупреждений в функции astype
df = df[(df["children"] != -1) & (df["children"] != 20)].copy();

### Вывод

#### total_income:  
имеет пустые значение в тех же строках что и days_employed, но если days_employed в финальных расчетах не участвует, то пустые данные total_income не позволят ответить на один из вопросов. Однако мы не можем потерять около 10% данных которые могут быть использованы для ответов не связанных с доходами, поэтому разумным вариантом будет заполнить значение Nan нулями для удобства работы а при работе с доходами использовать новую переменную income_without_null в которой будет хранится dataframe без нулевых доходов.  
Вероятной причиной отсутствия данных days_employed одновременно с total_income это выдача кредитов по неполному набору документов (например микрофинансовые организации). Было бы интересно проанализировать взаимосвязь выдачи таких кредитов с наличием задолженностей по кредиту.  
#### children:
т.к. неинтерпретируемые значение в этой колонке имеют всего 123 записи, что около половины процента от общего количества строк, я думаю безопасно для дальнейших подсчетов будет избавится от этих значений.

### Замена типа данных

In [3]:
df["total_income"] = df["total_income"].astype('int64')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Т.к. значение total_income уже было числовым (float64) необходимости конвертировать строку нет. Достаточно отбросить дробную для удобства расчетов присвоив колонку к типу int при помощи метода astype(int)

### Обработка дубликатов

In [4]:
# для анализа дупликатов и проверки что данные обработаны добавим проверку до и после вызова метода drop_duplicates
print('Duplicates count before drop duplicated')
print(len(df[df.duplicated()]))
df.drop_duplicates(subset=None, keep='first', inplace=True)
print('Duplicates count drop duplicated')
print(len(df[df.duplicated()]))


Duplicates count before drop duplicated
54
Duplicates count drop duplicated
0


### Вывод
В списке предоставленным банком обнаружилась 54 дублирующие строки. Дубликаты встречаются только среди строк, для которых отсутствует значения total_income и days_employed. Предположительно дубликаты создались при аггрегации данных из разных источников, при это часть данных уже содержалась в обоих таблицах.  
Для избавления от дубликатов воспользуемся методом drop_duplicates сохранив первое вхождение, т.к. эти данные нам понадобятся для дальнейших расчетов

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
all_lems = []
#создаем леммы для колонки purpose
def create_lemms(purpose):    
    lemms = m.lemmatize(purpose)
    lemms = list(filter(lambda x: (x != '\n') & (x != ' '), lemms))
    return lemms
    
df["purpose_lemm"] = df["purpose"].apply(create_lemms)
df["purpose_lemm"].value_counts()

[автомобиль]                                  967
[свадьба]                                     792
[на, проведение, свадьба]                     768
[сыграть, свадьба]                            764
[операция, с, недвижимость]                   672
[покупка, коммерческий, недвижимость]         659
[покупка, жилье, для, сдача]                  650
[операция, с, жилье]                          647
[операция, с, коммерческий, недвижимость]     645
[жилье]                                       641
[покупка, жилье]                              640
[покупка, жилье, для, семья]                  637
[недвижимость]                                631
[строительство, собственный, недвижимость]    628
[операция, со, свой, недвижимость]            623
[строительство, жилой, недвижимость]          621
[строительство, недвижимость]                 619
[покупка, свой, жилье]                        619
[покупка, недвижимость]                       616
[ремонт, жилье]                               604


### Вывод

Для хранения лемматизированных значений выделена отдельная колонка purpose_lemm. Для каждого значения колонки purpose осуществим вызов метода create_lemms.

Полученные леммы для удобства избавим от пробелов и символа новой строки, и выведем количество их значений. Объединим категории коммерческой недвижимости с общей категории жилья для удобства расчетов  
Получим 4 основные категории, для которых люди берут кредиты: автомобиль, свадьба, недвижимость, образование.

### Категоризация данных

In [6]:

#отсечка на половине дохода  от среднего дохода
income_mean = income_without_null["total_income"].median()
# отсечка на четверть от среднего дохода
income_mean_quart = income_without_null[income_without_null["total_income"] < income_mean]["total_income"].median()
# отсечка на три четверти от среднего дохода
income_mean_third_quart = income_without_null[income_without_null["total_income"] > income_mean]["total_income"].median()



# создаем категории для колонки для колонки purpose_lemm
# проверим наличие ключевого слова в каждом из набор лем. в случае если ключевого слова не обнаружено вернем -1
def create_purposeType(purpose_lem):
    if ('свадьба' in purpose_lem):
        return 'свадьба'
    elif (('недвижимость' in purpose_lem) | ('жилье' in purpose_lem)):
        return 'недвижимость'
    elif ('автомобиль' in purpose_lem):
        return 'автомобиль'
    elif ('образование' in purpose_lem):
        return 'образование'
    else:
        return -1
# создаем типы доходов для колонки total_income
# выше мы получили среднее медианное значение от суммы доходов, а затем среднее медианное значение 
# для доходов выше и ниже от этого значения. Получим разделение доходов на 4 группы, в каждой из групп будет
# находится одинаковое кол-во значений, но в каждой из групп будет свой уровень заработка.
def create_incomeType(income):
    if (income < income_mean_quart):
        return 0
    elif (income < income_mean):
        return 1
    elif (income < income_mean_third_quart):
        return 2
    else:
        return 3
    
df["purpose_type"] = df["purpose_lemm"].apply(create_purposeType)



income_without_null["income_type"] = income_without_null["total_income"].apply(create_incomeType)

# количество необработанных лемм
print("Необработанных лемм:")
print(len(df[df["purpose_type"] == -1]))
print("Количество значений в каждой из групп")
print(income_without_null["income_type"].value_counts())

Необработанных лемм:
0
Количество значений в каждой из групп
3    4838
2    4838
1    4838
0    4837
Name: income_type, dtype: int64


### Вывод

#### Категоризация цели
Исследуя значения в колонке purpose можем выделить 4 основные цели на которые пользователи брали кредит: свадьба, недвижимость, образование, автомобиль. Из колонки purpose_lemm выделим новую колонку purpose_type, в которой будет хранится одна из основных целей. 
#### Категоризация доходов
Для категоризации доходов разобьем их на 4 группы доходов, таким образом, что в каждой из групп будет одинаковое количество значений. Выделим колонку income_type хранящую тип уровная доходов. Получим распределение, с пиком во второй группе.  При категоризации доходов используем dataframe без строк с пропущенным значением.
  
Для колонок children и family_status дополнительную категоризацию можно не проводить, т.к. значение в этих колонках уже разбиты на несколько основных значений.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# для расчетов доли выплаченных кредитов к доле невыплаченных идеально подходит функция mean, 
# т.к. строка с выплаченным кредитом имеет значение 1, а с невыплаченным 0, среднее арифметическое и будет 
# искомым отношением выплаченных к невыплаченным.
# также выведем сумму агрегированных данных, чтобы оценить количество данных
pd.pivot_table(df, index=['children'],  values = 'debt', aggfunc= ["count", "mean"])

,count,mean
,debt,debt
children,,
0,14107,0.075353
1,4809,0.092327
2,2052,0.094542
3,330,0.081818
4,41,0.097561
5,9,0.000000


### Вывод

По полученным данным можно смело судить, что наличие детей значительно влияет на риск наличия задолжностей по кредиту. При среднем 7.5 процентов проблемных кредитов при отсутствии детей риск повышается до 9.2 при наличии одного ребенка и 9.4 при 2. Данными по трем и более детей вероятно стоит пренебречь в связи с небольшой выборкой.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
pd.pivot_table(df, index=['family_status'],  values = 'debt', aggfunc= ["count", "mean"])

,count,mean
,debt,debt
family_status,,
Не женат / не замужем,2796,0.097639
в разводе,1189,0.070648
вдовец / вдова,951,0.066246
гражданский брак,4146,0.092861
женат / замужем,12266,0.075575


### Вывод

Согласно полученным данным у незамужних\неженатых выше уровень проблем с своевременными платежами. Пользователи находящиеся или бывшие в браке в целом лучше справляются с платежами. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
pd.pivot_table(income_without_null, index=['income_type'],  values = 'debt', aggfunc= ["count", "mean"])

,count,mean
,debt,debt
income_type,,
0,4837,0.079181
1,4838,0.087019
2,4838,0.088053
3,4838,0.070484


### Вывод

Как показывают данные люди с средним доходом (100.000 - 200.000) чаще оказываются в ситуации просроченного платежа. Возможно это связано с тем, что люди с доходами ниже среднего привыкли следить за личными финансами и обращаться за помощью к кредитам аккуратней.

- Как разные цели кредита влияют на его возврат в срок?

In [16]:
pd.pivot_table(df, index=['purpose_type'],  values = 'debt', aggfunc= ["count", "mean"])

,count,mean
,debt,debt
purpose_type,,
автомобиль,4281,0.093436
недвижимость,10754,0.072531
образование,3989,0.092504
свадьба,2324,0.078744


### Вывод

Наиболее безопасным с точки зрения просроченных платежей является кредит на недвижимость. Кредиты на недвижимость обычно достаточно большие и берутся на десятки лет, люди относятся к ним осмотрительней. К тому же с собственностью с меньшей вероятностью произойдет какая либо неприятность (в отличии от автомобиля который могут например угнать) 
Кредиты на свадьбу оказался на уровне средней задолжности.
Кредиты же на автомобиль и на образования оказались наиболее рискованными

### Шаг 4. Общий вывод

На основе проведенного исследования можно сделать следующие выводы:  
1) Наличие даже одного ребенка повышает шанс на задержку в выплатах кредита  
2) Наиболее ответственные кредиторы это состоящие или бывшие в браке  
3) Люди с средними доходами чаще остальных групп населения задерживают платежи  
4) Самые безопасные кредиты - кредиты на недвижимость, а небезопасные на образование и автомобиль.  